# XML Testing

Author @ysbecca

In [1]:
import os
import os
OPENSLIDE_PATH = r"E:\Java\openslide-win64-20221217\bin"
os.add_dll_directory(OPENSLIDE_PATH)
from xml.dom import minidom
import numpy as np
import openslide
from openslide import open_slide 
from openslide.deepzoom import DeepZoomGenerator
from glob import glob

In [2]:
path = r'H:\ICIAR2018BACH_Challenge\ICIAR2018_BACH_Challenge\py_wsi_data\A02.xml'

In [3]:
xml = minidom.parse(path)
# The first region marked is always the tumour delineation
regions_ = xml.getElementsByTagName("Region")
regions, region_labels = [], []
for region in regions_:
    vertices = region.getElementsByTagName("Vertex")
    attribute = region.getElementsByTagName("Attribute")
    if len(attribute) > 0:
        r_label = attribute[0].attributes['Value'].value
    else:
        r_label = region.getAttribute('Text')
    region_labels.append(r_label)
    
    # Store x, y coordinates into a 2D array in format [x1, y1], [x2, y2], ...
    coords = np.zeros((len(vertices), 2))
    
    for i, vertex in enumerate(vertices):
        coords[i][0] = vertex.attributes['X'].value
        coords[i][1] = vertex.attributes['Y'].value
        
    regions.append(coords)

In [14]:
def get_dimensions(lst):
    if isinstance(lst, list):
        # 如果是列表，递归检查第一个元素的维度
        return [len(lst)] + get_dimensions(lst[0])
    else:
        # 如果不是列表，返回空列表表示维度结束
        return []
    
print(get_dimensions(regions))
print(get_dimensions(region_labels))
# regions_nparray=np.array(regions)
# region_labels_nparray = np.array(region_labels)
# print(regions_nparray.shape)
# print(region_labels_nparray.shape)

# print(np.shape(regions))
# print(np.shape(region_labels))

[17]
[17]


In [16]:
region_labels
# print(len(region_labels))
# print(region_labels)

['Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Benign',
 'Carcinoma in situ',
 'Carcinoma in situ',
 'Carcinoma invasive',
 'Carcinoma invasive',
 'Carcinoma invasive']

## Determine whether a point is within a polygon given its vertices.
## 判断一个点是否在给定顶点的多边形内,通过判断点是否在某个多边形中从而判断点的label

In [12]:
from shapely.geometry import Polygon, Point

In [30]:
label_map = {'Normal': 0,
             'Benign': 1,
             'Carcinoma in situ': 2,
             'Carcinoma invasive': 3,
            }


def generate_label(regions, region_labels, point):
    # regions = array of vertices (all_coords)
    # point [x, y]
    for i in range(len(region_labels)):
        poly = Polygon(regions[i])
        if poly.contains(Point(point[0], point[1])):
            return label_map[region_labels[i]]
    return label_map['Normal']

In [19]:
generate_label(regions, region_labels, [7500, 21600])

0

In [20]:
patch_size = 256
percent_overlap = 0
file_dir = r"H:\ICIAR2018BACH_Challenge\ICIAR2018_BACH_Challenge\py_wsi_data"
file_name = "A01.svs"
xml_file = "A01.xml"
xml_dir = r"H:\ICIAR2018BACH_Challenge\ICIAR2018_BACH_Challenge\py_wsi_data"
level = 12

In [22]:
overlap = int(patch_size*percent_overlap / 2.0)
tile_size = patch_size - overlap*2

slide = open_slide(file_dir +"\\" + file_name) 
tiles = DeepZoomGenerator(slide, tile_size=tile_size, overlap=overlap, limit_bounds=False)

if level >= tiles.level_count:
    print("Error: requested level does not exist. Slide level count: " + str(tiles.level_count))

x_tiles, y_tiles = tiles.level_tiles[level]

In [24]:
print(x_tiles)
print(y_tiles)

16
11


In [27]:
tiles.get_tile_coordinates(level, (5, 2))[0]
# tiles.get_tile_coordinates(level,(5,2))[1]

(20480, 8192)

In [31]:
patches, coords, labels = [], [], []
x, y = 0, 0
count = 0
while y < y_tiles:
    while x < x_tiles:
        new_tile = np.array(tiles.get_tile(level, (x, y)), dtype=np.int8)
        # OpenSlide calculates overlap in such a way that sometimes depending on the dimensions, edge 
        # patches are smaller than the others. We will ignore such patches.
        if np.shape(new_tile) == (patch_size, patch_size, 3):
            patches.append(new_tile)
            coords.append(np.array([x, y]))
            count += 1

            # Calculate the patch label based on centre point.
            if xml_file:
                converted_coords = tiles.get_tile_coordinates(level, (x, y))[0]
                labels.append(generate_label(regions, region_labels, converted_coords))
        x += 1
    y += 1
    x = 0

# image_ids = [im_id]*count

In [34]:
print(np.shape(patches))
print(np.shape(coords))
print(np.shape(labels))

(150, 256, 256, 3)
(150, 2)
(150,)


In [35]:
np.count_nonzero(labels)

38

In [36]:
name = 'tester.svs'

In [37]:
name[:-4]

'tester'